In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uproot
import utils.hist_utils as hu # functions created by me
from importlib import reload
import ROOT

In [97]:
reload(hu)
hu.test_utils()

hist_utils.py imported successfully :)
In order to update it more dinamically, use 'reload' method from 'importlib' library.


In [98]:
def get_hist(filePath, rootDir_name, histName):

    """
    Get histogram from root file
    """
    root_file = ROOT.TFile.Open(filePath)
    rootDir = root_file.Get(rootDir_name) # can be e.g "dir/subdir/subdir"
    if not rootDir:
        print(f"{rootDir_name} not found in {filePath}.")
    mcSigHist = rootDir.FindObject(histName)
    if not mcSigHist:
        print(f"{histName} not found in {rootDir_name}")
    return mcSigHist
def join_dfs(dir_path, fileNames, rootDir_name, histName, datasetsNames=None):

    """
    Create dataframes from histograms from many root files and joins them into a single dataframe.
    """
    dfs = pd.DataFrame()
    for i,fileName in enumerate(fileNames):
        filePath = f"{dir_path}/{fileName}"
        hist = get_hist(filePath, rootDir_name, histName)
        df = hu.hist_to_df(hist, isLabeledHist=True)
        if i==0:
            dfs["XLabel"] = df["XLabel"]
            dfs["YLabel"] = df["YLabel"]
            print(dfs.to_string())
        if datasetsNames is not None:
             dfs[datasetsNames[i]] = df["counts"]
        else:
             dfs[fileName] = df["counts"]
    return dfs

In [8]:
file = uproot.open("/home/ferrandi/alice/skimming/AnalysisResultsTestMakerNoPID14.root")
# file.keys()

In [9]:
dir_path = '~/alice/skimming'
fileName = 'AnalysisResultsTestMakerNoPID14.root'
filePath = f"{dir_path}/{fileName}"
rootDir_name = 'table-maker-m-c/Statistics'
histName = "MCsignals"
mcSigHist = get_hist(filePath, rootDir_name, histName)

In [10]:
datasets = np.array(['14', '15', '16', '17'])
fileNamesNoPID = 'AnalysisResultsTestMakerNoPID'+datasets+'.root'
fileNamesPID = 'AnalysisResultsTestMakerWithPID'+datasets+'.root'
print(fileNamesNoPID)
print(fileNamesPID)

['AnalysisResultsTestMakerNoPID14.root'
 'AnalysisResultsTestMakerNoPID15.root'
 'AnalysisResultsTestMakerNoPID16.root'
 'AnalysisResultsTestMakerNoPID17.root']
['AnalysisResultsTestMakerWithPID14.root'
 'AnalysisResultsTestMakerWithPID15.root'
 'AnalysisResultsTestMakerWithPID16.root'
 'AnalysisResultsTestMakerWithPID17.root']


In [11]:
# c1 = ROOT.TCanvas("c1", "MC Signal Histogram", 800, 600)
# mcSigHist.Draw()
# c1.Draw()

In [12]:
dfMCSig = hu.hist_to_df(mcSigHist, isLabeledHist=True)
dfMCSig
dftest=pd.DataFrame()
dftest["Label"]=dfMCSig["Label"]
dftest

,Label
0,eFromJpsi
1,eFromPromptJpsi
2,eFromNonpromptJpsi
3,Jpsi
4,nonPromptJpsi
5,promptJpsi
6,allBeautyHadrons
7,Bplus
8,protonPrimary
9,Others (matched to reco tracks)


In [13]:
dir_path = '~/alice/skimming'
rootDir_name = 'table-maker-m-c/Statistics'
histName = "MCsignals"
datasetsNames = "LHC25b"+np.array(datasets)
dfMCSigsPID = join_dfs(dir_path, fileNamesPID, rootDir_name, histName, datasetsNames)
dfMCSigsNoPID = join_dfs(dir_path, fileNamesNoPID, rootDir_name, histName, datasetsNames)
print(dfMCSigsPID, "\n")
print(dfMCSigsNoPID)


                             Label  LHC25b14  LHC25b15  LHC25b16  LHC25b17
0                        eFromJpsi   52270.0   81998.0   49250.0   96356.0
1                  eFromPromptJpsi   42728.0      32.0   40114.0      26.0
2               eFromNonpromptJpsi    9542.0   81966.0    9136.0   96330.0
3                             Jpsi   26135.0   41236.0   24625.0   48428.0
4                    nonPromptJpsi    4266.0   40983.0    4091.0   48165.0
5                       promptJpsi   21364.0     253.0   20057.0     263.0
6                 allBeautyHadrons   27736.0  112489.0   26447.0  132348.0
7                            Bplus    7060.0   26634.0    6674.0   31764.0
8                    protonPrimary  837371.0  824057.0  788726.0  969186.0
9  Others (matched to reco tracks)  105759.0  102640.0   98793.0   83813.0 

                             Label  LHC25b14  LHC25b15  LHC25b16  LHC25b17
0                        eFromJpsi   45196.0   71308.0   43074.0   83428.0
1                  eFro

In [111]:
dfEffMCSigsPID = dfMCSigsPID.copy()
dfEffMCSigsPID.iloc[:,1:] = dfMCSigsNoPID.iloc[:,1:]/dfEffMCSigsPID.iloc[:,1:]
dfEffMCSigsPID

,Label,LHC25b14,LHC25b15,LHC25b16,LHC25b17
0,eFromJpsi,0.864664,0.869631,0.874599,0.865831
1,eFromPromptJpsi,0.862947,0.937500,0.874956,0.923077
2,eFromNonpromptJpsi,0.872354,0.869604,0.873030,0.865815
3,Jpsi,0.864664,0.869531,0.874599,0.865821
4,nonPromptJpsi,0.874824,0.869604,0.871670,0.865815
5,promptJpsi,0.862947,0.857708,0.874956,0.866920
6,allBeautyHadrons,0.871178,0.869881,0.876432,0.867455
7,Bplus,0.868555,0.871856,0.878634,0.864438
8,protonPrimary,0.864924,0.870070,0.875692,0.869442
9,Others (matched to reco tracks),2.232548,2.214439,2.357637,2.368344


In [99]:
dir_path = '~/alice/skimming'
rootDir_name = 'table-maker-m-c/Statistics'
histName = "EventStats"
datasetsNames = "LHC25b"+np.array(datasets)
dfEvtStatsPID = join_dfs(dir_path, fileNamesPID, rootDir_name, histName, datasetsNames)
dfEvtStatsNoPID = join_dfs(dir_path, fileNamesNoPID, rootDir_name, histName, datasetsNames)
# print(dfEvtStatsPID.to_string())
# for i in dfEvtStatsPID(dfEvtStatsPID["XLabel"]=="Before cuts"):
#     print("\n",i)

                          XLabel                      YLabel
0                            BCs                    kIsBBV0A
1                            BCs                    kIsBBV0C
2                            BCs                    kIsBBFDA
3                            BCs                    kIsBBFDC
4                            BCs                    kIsBBT0A
5                            BCs                    kIsBBT0C
6                            BCs                    kNoBGV0A
7                            BCs                    kNoBGV0C
8                            BCs                    kNoBGFDA
9                            BCs                    kNoBGFDC
10                           BCs                    kNoBGT0A
11                           BCs                    kNoBGT0C
12                           BCs                    kIsBBZNA
13                           BCs                    kIsBBZNC
14                           BCs                    kIsBBZAC
15                      

In [133]:
dfEvtStatsPIDAfterCuts = dfEvtStatsPID.query("XLabel == 'After cuts' and YLabel == 'Total' ")
dfEvtStatsPIDBeforeCuts = dfEvtStatsPID.query("XLabel == 'Before cuts' and YLabel == 'Total' ")
#and YLabel == 'Total' ")
dfEvtStatsNoPIDAfterCuts = dfEvtStatsNoPID.query("XLabel == 'After cuts' and YLabel == 'Total' ")
dfEvtStatsNoPIDBeforeCuts = dfEvtStatsNoPID.query("XLabel == 'Before cuts' and YLabel == 'Total' ")

print(dfEvtStatsPIDAfterCuts, "\n")
print(dfEvtStatsPIDBeforeCuts, "\n")
print(dfEvtStatsNoPIDAfterCuts, "\n")
print(dfEvtStatsNoPIDBeforeCuts, "\n")
print("\n")

dfEffEvtStatsPID = dfEvtStatsPIDAfterCuts.copy()
dfEffEvtStatsPID.iloc[:, 2:] = (
    dfEffEvtStatsPID.iloc[:, 2:] /
    dfEvtStatsPIDBeforeCuts.iloc[:, 2:].values
)
dfEffEvtStatsPID["XLabel"] = "Cuts Efficiency With PID"
dfEffEvtStatsPID.drop(columns=["YLabel"], inplace=True)
print(dfEffEvtStatsPID)

dfEffEvtStatsNoPID = dfEvtStatsNoPIDAfterCuts.copy()
dfEffEvtStatsNoPID.iloc[:, 2:] = (
    dfEffEvtStatsNoPID.iloc[:, 2:] /
    dfEvtStatsNoPIDBeforeCuts.iloc[:, 2:].values
)
dfEffEvtStatsNoPID["XLabel"] = "Cuts Efficiency W/o PID"
dfEffEvtStatsNoPID.drop(columns=["YLabel"], inplace=True)
print(dfEffEvtStatsNoPID)

         XLabel YLabel  LHC25b14  LHC25b15  LHC25b16  LHC25b17
203  After cuts  Total   80424.0   83624.0   74187.0   68730.0 

          XLabel YLabel  LHC25b14  LHC25b15  LHC25b16  LHC25b17
152  Before cuts  Total  115517.0  119517.0  108640.0   99679.0 

         XLabel YLabel  LHC25b14  LHC25b15  LHC25b16  LHC25b17
203  After cuts  Total   69752.0   72742.0   64941.0   60287.0 

          XLabel YLabel  LHC25b14  LHC25b15  LHC25b16  LHC25b17
152  Before cuts  Total  100018.0  103921.0   94958.0   87301.0 



                       XLabel  LHC25b14  LHC25b15  LHC25b16  LHC25b17
203  Cuts Efficiency With PID  0.696209  0.699683   0.68287  0.689513
                      XLabel  LHC25b14  LHC25b15  LHC25b16  LHC25b17
203  Cuts Efficiency W/o PID  0.697394  0.699974  0.683892  0.690565
